In [3]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import os

os.environ["KERAS_BACKEND"] = "plaidml.bridge.keras"

In [8]:
df= pd.read_csv('Dataset/Churn_Modelling.csv')

df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Preprocess

In [9]:
# Seleksi fitur
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace= True)

# Categorical encoding
df= df.replace({'Female':0, 'Male':1})
df= pd.get_dummies(df, columns=['Geography'])

# Pindahkan kolom target ke kanan
exited_cols= df['Exited']
df.drop('Exited', axis=1, inplace=True)
df['Exited']= exited_cols

# Normalisasi
from sklearn.preprocessing import MinMaxScaler

scaler= MinMaxScaler()
cols_to_scale= [i for i in df if df[i].max()!=1]

df[cols_to_scale]= scaler.fit_transform(df[cols_to_scale])

df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Exited
0,0.538,0,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,0,0,1
1,0.516,0,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,1,0
2,0.304,0,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,0,0,1
3,0.698,0,0.283784,0.1,0.000000,0.333333,0,0,0.469120,1,0,0,0
4,1.000,0,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.842,1,0.283784,0.5,0.000000,0.333333,1,0,0.481341,1,0,0,0
9996,0.332,1,0.229730,1.0,0.228657,0.000000,1,1,0.508490,1,0,0,0
9997,0.718,0,0.243243,0.7,0.000000,0.000000,0,1,0.210390,1,0,0,1
9998,0.844,1,0.324324,0.3,0.299226,0.333333,1,0,0.464429,0,1,0,1


Train test split

In [10]:
x= df.drop('Exited', axis=1)
y= df['Exited']

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.2)

### Batch Size pada proses training

Batch Size merupakan seberapa banyak data yang dimasukkan ke dalam neural network saat proses train per iterasi, jika tidak mendefinisikan batch size, maka jumlah data yang dimasukkan adalah 1. Semakin tinggi ukuran batch, semakin cepat proses training, namun semakin banyak ruang memori yang  diperlukan dan sedikit merusak model.
<br><br>
Untuk mendapatkan ukuran batch yang optimal biasanya dengan proses trial and error, namun umumnya disarankan untuk mencoba ukuran batch yang lebih kecil terlebih dahulu (biasanya 32 atau 64), ukuran batch yang kecil memerlukan learning rate yang kecil. Jumlah ukuran batch juga disarankan merupangkat pangkat dari 2 untuk mendapatkan keuntungan penuh dari pemrosesan GPU 

Misal modelnya seperti ini

In [24]:
model= keras.Sequential([
    keras.layers.Dense(256, input_shape=(len(x_train.columns),), activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='tanh'),
    keras.layers.Dense(1, activation='sigmoid')
])

loss= keras.losses.BinaryCrossentropy(from_logits=False)
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

Kita mendefinisikan batch_size saat proses fit(), kita juga akan membandingkan waktu train dengan dan tanpa batch size

In [27]:
import time

start= time.time()
model.fit(x_train, y_train, epochs=10, batch_size=100)
print(f'training time= {time.time()-start}')

Epoch 1/10
80/80 [==============================] - 0s 1ms/step - loss: 0.4837 - accuracy: 0.7896
Epoch 2/10
80/80 [==============================] - 0s 1ms/step - loss: 0.4350 - accuracy: 0.8096
Epoch 3/10
80/80 [==============================] - 0s 1ms/step - loss: 0.4097 - accuracy: 0.8286
Epoch 4/10
80/80 [==============================] - 0s 1ms/step - loss: 0.3844 - accuracy: 0.8391
Epoch 5/10
80/80 [==============================] - 0s 1ms/step - loss: 0.3629 - accuracy: 0.8496
Epoch 6/10
80/80 [==============================] - 0s 1ms/step - loss: 0.3485 - accuracy: 0.8570
Epoch 7/10
80/80 [==============================] - 0s 1ms/step - loss: 0.3439 - accuracy: 0.8559
Epoch 8/10
80/80 [==============================] - 0s 1ms/step - loss: 0.3413 - accuracy: 0.8595
Epoch 9/10
80/80 [==============================] - 0s 1ms/step - loss: 0.3386 - accuracy: 0.8616
Epoch 10/10
80/80 [==============================] - 0s 1ms/step - loss: 0.3309 - accuracy: 0.8644
training time= 1.40

In [23]:
model.evaluate(x_test, y_test)

63/63 [==============================] - 0s 755us/step - loss: 0.3449 - accuracy: 0.8600


[0.34490156173706055, 0.8600000143051147]

In [28]:
start= time.time()
model.fit(x_train, y_train, epochs=10)
print(f'training time= {time.time()-start}')

Epoch 1/10
250/250 [==============================] - ETA: 0s - loss: 0.3461 - accuracy: 0.85 - 1s 1ms/step - loss: 0.3470 - accuracy: 0.8584
Epoch 2/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3383 - accuracy: 0.8604: 0s - loss: 0.3359 - accuracy: 0.86 - ETA: 0s - loss: 0.3370 - accura
Epoch 3/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3331 - accuracy: 0.8649
Epoch 4/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3335 - accuracy: 0.8651
Epoch 5/10
250/250 [==============================] - 0s 2ms/step - loss: 0.3310 - accuracy: 0.8635
Epoch 6/10
250/250 [==============================] - 0s 2ms/step - loss: 0.3301 - accuracy: 0.8620
Epoch 7/10
250/250 [==============================] - 0s 2ms/step - loss: 0.3276 - accuracy: 0.8635
Epoch 8/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3242 - accuracy: 0.8691
Epoch 9/10
250/250 [==============================] - 0s 2ms/step - loss: 0.3233 - accur

In [29]:
model.evaluate(x_test, y_test)

63/63 [==============================] - 0s 862us/step - loss: 0.3417 - accuracy: 0.8590


[0.34168344736099243, 0.859000027179718]

Kita bisa melihat model dengan data yang ditrain per batch mempunyai proses train yang lebih cepat daripada yang tidak

### Batch Normalization

Batch Normalization singkatnya merupakan proses normalisasi terhadap output dari layer sebelumnya. Mengapa dilakukan normalisasi? biasanya output dari suatu layer mempunyai rentang yang besar (walaupun inputnya sudah ternormalisasi terlebih dahulu), ini menyebabkan proses feed ke layer selanjutnya kurang baik dan model tidak general, maka dilakukan normalisasi untuk men-skala output menjadi lebih seimbang. Batch Normalization juga mempunyai <a href='https://www.analyticsvidhya.com/blog/2021/03/introduction-to-batch-normalization/'>Manfaat lainnya</a>
<br><br>
Mengapa dilakukan per-batch? (nda tau)
<br><br>
Batch Normalization pada model deep learning berupa layer, misal diletakkan layer batch normalization pada layer terakhir sebelum output, berarti layer output akan menerima inputan yang telah dinormalisasi

In [33]:
model= keras.Sequential([
    keras.layers.Dense(256, input_shape=(len(x_train.columns),), activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='tanh'),
    keras.layers.BatchNormalization(axis=1),
    keras.layers.Dense(1, activation='sigmoid')
])

loss= keras.losses.BinaryCrossentropy(from_logits=False)
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [35]:
model.fit(x_train, y_train, epochs=10, batch_size= 32)

Epoch 1/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3434 - accuracy: 0.8569
Epoch 2/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3371 - accuracy: 0.8612
Epoch 3/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3369 - accuracy: 0.8625
Epoch 4/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3341 - accuracy: 0.8601
Epoch 5/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3384 - accuracy: 0.8572
Epoch 6/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3344 - accuracy: 0.8631
Epoch 7/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3311 - accuracy: 0.8633: 0s - loss: 0.3189 - accuracy: 
Epoch 8/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3305 - accuracy: 0.8670
Epoch 9/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3313 - accuracy: 0.8654
Epoch 10/10
250/250 [==============================] - 0s 1ms/step -

In [36]:
model.evaluate(x_test, y_test)

63/63 [==============================] - 0s 862us/step - loss: 0.3502 - accuracy: 0.8545


[0.3502262234687805, 0.8544999957084656]